# Using Conditional Random Fields and Python for Latin word segmentation

In this project, most texts from the Latin Library are being utilized to train a CRF the segmentation of Latin texts. 
For several centuries, Latin text was written without the use of space characters or any other word delimiters (scripio continua).

First, perform some imports. The Python CRFSuite can be installed via
__ pip install python-crfsuite __

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, HTTPError
import pycrfsuite

In [2]:
base_url = "http://www.thelatinlibrary.com/"

## Retrieving our training data

Now, get most links on the Latin Library's homepage -- ignoring some links that are not associated with a particular author.

In [3]:
home_content = urlopen(base_url)
soup = BeautifulSoup(home_content, "lxml")
author_page_links = soup.find_all("a")
author_pages = [ap["href"] for i, ap in enumerate(author_page_links) if i < 49]
ap_content = list()
texts = list()

In [4]:
for ap in author_pages:
    ap_content.append(urlopen(base_url + ap))

Next, create a list of all links pointing to Latin texts. The Latin Library uses a special format which makes it easy to find the corresponding links: All of these links contain the name of the text author.

In [5]:
book_links = list()

In [6]:
for path, content in zip(author_pages, ap_content):
    author_name = path.split(".")[0]
    ap_soup = BeautifulSoup(content, "lxml")
    book_links += ([link for link in ap_soup.find_all("a", {"href": True}) if author_name in link["href"]])

print(book_links[0])

<a href="ammianus/14.shtml">Liber XIV</a>


Get the text content and write it to a list. We will not need all of the books available, just take the first 200 pages.

In [7]:
texts = list()
num_pages = 200

for i, bl in enumerate(book_links[:num_pages]):
    print("Getting content " + str(i + 1) + " of " + str(num_pages), end="\r", flush=True)
    try:
        content = urlopen(base_url + bl["href"]).read() 
        texts.append(content)
    except HTTPError as err:
        print("Unable to retrieve " + bl["href"] + ".")
        continue

The text that we would like to retrieve is written on every book page in its paragraphs __1__ to __-1__. 
Then, split the text at periods to convert it into sentences which we will use for training later on.

In [8]:
sentences = list()

for i, text in enumerate(texts):
    print("Document " + str(i + 1) + " of " + str(len(texts)), end="\r", flush=True)
    textSoup = BeautifulSoup(text, "lxml")
    paragraphs = textSoup.find_all("p", attrs={"class":None})
    prepared = ("".join([p.text.strip().lower() for p in paragraphs[1:-1]]))
    for t in prepared.split("."):
        part = "".join([c for c in t if c.isalpha() or c.isspace()])
        sentences.append(part.strip())

print(sentences[200])

infamabat autem haec suspicio latinum domesticorum comitem et agilonem tribunum stabuli atque scudilonem scutariorum rectorem qui tunc ut dextris suis gestantes rem publicam colebantur


In [9]:
sentences = [s for s in sentences if len(s) > 5] # remove very short "sentences"

In [10]:
print(sentences[200])

tentis igitur regis utriusque legatis et negotio tectius diu pensato cum pacem oportere tribui quae iustis condicionibus petebatur eamque ex re tum fore sententiarum via concinens adprobasset advocato in contionem exercitu imperator pro tempore pauca dicturus tribunali adsistens circumdatus potestatum coetu celsarum ad hunc disservit modum nemo quaeso miretur si post exsudatos labores itinerum longos congestosque adfatim commeatus fiducia vestri ductante barbaricos pagos adventans velut mutato repente consilio ad placidiora deverti


For preparing our training data, every sentence is converted into a char list together with the information wether the char marks the beginning of a new word.

In [11]:
prepared_sentences = list()

for sentence in sentences:
    lengths = [len(w) for w in sentence.split(" ")]
    positions = []

    next_pos = 0
    for length in lengths:
        next_pos = next_pos + length
        positions.append(next_pos)
    concatenated = sentence.replace(" ", "")

    chars = [c for c in concatenated]
    labels = [0 if not i in positions else 1 for i, c in enumerate(concatenated)]

    prepared_sentences.append(list(zip(chars, labels)))
    
    
print([d for d in prepared_sentences[200]])

[('t', 0), ('e', 0), ('n', 0), ('t', 0), ('i', 0), ('s', 0), ('i', 1), ('g', 0), ('i', 0), ('t', 0), ('u', 0), ('r', 0), ('r', 1), ('e', 0), ('g', 0), ('i', 0), ('s', 0), ('u', 1), ('t', 0), ('r', 0), ('i', 0), ('u', 0), ('s', 0), ('q', 0), ('u', 0), ('e', 0), ('l', 1), ('e', 0), ('g', 0), ('a', 0), ('t', 0), ('i', 0), ('s', 0), ('e', 1), ('t', 0), ('n', 1), ('e', 0), ('g', 0), ('o', 0), ('t', 0), ('i', 0), ('o', 0), ('t', 1), ('e', 0), ('c', 0), ('t', 0), ('i', 0), ('u', 0), ('s', 0), ('d', 1), ('i', 0), ('u', 0), ('p', 1), ('e', 0), ('n', 0), ('s', 0), ('a', 0), ('t', 0), ('o', 0), ('c', 1), ('u', 0), ('m', 0), ('p', 1), ('a', 0), ('c', 0), ('e', 0), ('m', 0), ('o', 1), ('p', 0), ('o', 0), ('r', 0), ('t', 0), ('e', 0), ('r', 0), ('e', 0), ('t', 1), ('r', 0), ('i', 0), ('b', 0), ('u', 0), ('i', 0), ('q', 1), ('u', 0), ('a', 0), ('e', 0), ('i', 1), ('u', 0), ('s', 0), ('t', 0), ('i', 0), ('s', 0), ('c', 1), ('o', 0), ('n', 0), ('d', 0), ('i', 0), ('c', 0), ('i', 0), ('o', 0), ('n', 0),

## Transforming the characters to feature vectors.

Finally, we can create some simple n-gram features. Obviously, you could think of much more sophisticated features and possibly improve our model's performance.

In [12]:
def create_char_features(sentence, i):
    features = [
        'bias',
        'char=' + sentence[i][0] 
    ]
    
    if i >= 1:
        features.extend([
            'char-1=' + sentence[i-1][0],
            'char-1:0=' + sentence[i-1][0] + sentence[i][0],
        ])
    else:
        features.append("BOS")
        
    if i >= 2:
        features.extend([
            'char-2=' + sentence[i-2][0],
            'char-2:0=' + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-2:-1=' + sentence[i-2][0] + sentence[i-1][0],
        ])
        
    if i >= 3:
        features.extend([
            'char-3:0=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-3:-1=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0],
        ])
        
        
    if i + 1 < len(sentence):
        features.extend([
            'char+1=' + sentence[i+1][0],
            'char:+1=' + sentence[i][0] + sentence[i+1][0],
        ])
    else:
        features.append("EOS")
        
    if i + 2 < len(sentence):
        features.extend([
            'char+2=' + sentence[i+2][0],
            'char:+2=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0],
            'char+1:+2=' + sentence[i+1][0] + sentence[i+2][0],
        ])
        
    if i + 3 < len(sentence):
        features.extend([
            'char:+3=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0]+ sentence[i+3][0],
            'char+1:+3=' + sentence[i+1][0] + sentence[i+2][0] + sentence[i+3][0],
        ])
    
    return features



def create_sentence_features(prepared_sentence):
    return [create_char_features(prepared_sentence, i) for i in range(len(prepared_sentence))]

def create_sentence_labels(prepared_sentence):
    return [str(part[1]) for part in prepared_sentence]

In [13]:
X = [create_sentence_features(ps) for ps in prepared_sentences[:-10000]]
y = [create_sentence_labels(ps)   for ps in prepared_sentences[:-10000]]

X_test = [create_sentence_features(ps) for ps in prepared_sentences[-10000:]]
y_test = [create_sentence_labels(ps)   for ps in prepared_sentences[-10000:]]

## Training a CRF
Now, we use Python-CRFSuite for training a CRF.

In [14]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X, y):
    trainer.append(xseq, yseq)

In [15]:
trainer.set_params({
    'c1': 1.0, 
    'c2': 1e-3,
    'max_iterations': 60,
    'feature.possible_transitions': True
})

In [16]:
trainer.train('latin-text-segmentation.crfsuite')

In [17]:
tagger = pycrfsuite.Tagger()
tagger.open('latin-text-segmentation.crfsuite')

In [18]:
def segment_sentence(sentence):
    sent = sentence.replace(" ", "")
    prediction = tagger.tag(create_sentence_features(sent))
    complete = ""
    for i, p in enumerate(prediction):
        if p == "1":
            complete += " " + sent[i]
        else:
            complete += sent[i]
    return complete

In [19]:
print(segment_sentence("dominusadtemplumproperat"))
print(segment_sentence("portapatet"))

dominus ad templum properat
porta patet


Finally, let's find out how well our model performs.

In [20]:
tp = 0
fp = 0
fn = 0
n_correct = 0
n_incorrect = 0

for s in prepared_sentences[-10000:]:
    prediction = tagger.tag(create_sentence_features(s))
    correct = create_sentence_labels(s)
    zipped = list(zip(prediction, correct))
    tp +=        len([_ for l, c in zipped if l == c and l == "1"])
    fp +=        len([_ for l, c in zipped if l == "1" and c == "0"])
    fn +=        len([_ for l, c in zipped if l == "0" and c == "1"])
    n_incorrect += len([_ for l, c in zipped if l != c])
    n_correct   += len([_ for l, c in zipped if l == c])

In [21]:
print("Precision:\t" + str(tp/(tp+fp)))
print("Recall:\t\t" + str(tp/(tp+fn)))
print("Accuracy:\t" + str(n_correct/(n_correct+n_incorrect)))

Precision:	0.9314353553833713
Recall:		0.9171904737701122
Accuracy:	0.9766116709677363
